In [1]:
import requests
from bs4 import BeautifulSoup


In [28]:

def get_author_profile(researcher_id):
    url = f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        author_name = extract_author_name(soup)
        print(f"Author Name: {author_name}")
        
        affiliation = extract_affiliation(soup)
        print(f"Affiliation: {affiliation}")
        
        sinta_score = extract_sinta_score(soup)
        if sinta_score:
            print(f"SINTA Score Overall: {sinta_score}")
        
        subject_list = extract_subjects(soup)
        print(f"Subjects: {', '.join(subject_list)}")
        
        total_pages = extract_total_pages(soup)
        if total_pages:
            print(f"Total Pages: {total_pages}")
            get_all_scopus_documents(researcher_id, total_pages)
    else:
        print(f"Failed to retrieve page with status code: {response.status_code}")

def extract_author_name(soup):
    return soup.find('h3').get_text(strip=True)

def extract_affiliation(soup):
    return soup.find('a', href=lambda x: x and 'affiliations' in x).get_text(strip=True)

def extract_sinta_score(soup):
    sinta_score_div = soup.find('div', class_='pr-num')
    if sinta_score_div:
        return sinta_score_div.get_text(strip=True)
    return None

def extract_subjects(soup):
    subject_list = []
    subjects = soup.find_all('li', class_='subject-list')
    for subject in subjects:
        subject_list.append(subject.get_text(strip=True))
    return subject_list


In [36]:

def extract_total_pages(soup):
   # Mencari elemen pagination
    pagination = soup.find("ul", class_="pagination justify-content-center")
    
    if pagination:
        # Mengambil semua elemen 'page-item' di dalam pagination
        pages = pagination.find_all("li", class_="page-item")
        
        # Menghitung jumlah halaman
        # Mengabaikan 'Previous' dan 'Next' dengan menyaring page-item yang berisi angka
        total_pages = sum(1 for page in pages if page.text.isdigit())
        print(f"Total halaman publikasi: {total_pages}")
    else:
        print("Informasi jumlah halaman tidak ditemukan.")

def get_all_scopus_documents(researcher_id, total_pages):
    publications = []
    for page_num in range(1, total_pages + 1):
        page_url = f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}"
        page_response = requests.get(page_url)
        if page_response.status_code == 200:
            page_soup = BeautifulSoup(page_response.content, 'html.parser')
            publications.extend(extract_publications(page_soup))
        else:
            print(f"Failed to retrieve page {page_num} with status code: {page_response.status_code}")
    
    for idx, title in enumerate(publications, 1):
        print(f"Publication {idx}: {title}")

def extract_publications(soup):
    publications = soup.find_all('div', class_='ar-title')
    return [publication.get_text(strip=True) for publication in publications]



In [40]:
url = "https://sinta.kemdikbud.go.id/authors/profile/5986966"

try:
    # Mengirim permintaan ke server
    response = requests.get(url)
    response.raise_for_status()  # Memastikan respons yang sukses (status 200)
    
    # Mem-parsing konten HTML dengan BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Memeriksa elemen pagination
    pagination = soup.find("ul", class_="pagination justify-content-center")
    
    if pagination:
        print("Halaman ini memiliki elemen pagination.")
        
        # Menghitung jumlah halaman dari elemen pagination
        pages = pagination.find_all("li", class_="page-item")
        total_pages = sum(1 for page in pages if page.text.isdigit())
        print(f"Total halaman publikasi: {total_pages}")
    else:
        print("Halaman ini tidak memiliki elemen pagination.")
        
except requests.exceptions.RequestException as e:
    print(f"Gagal mengakses halaman: {e}")

Halaman ini tidak memiliki elemen pagination.


In [37]:

# Call the function with the researcher_id
get_author_profile(5986966)


Author Name: AJI SURAJI
Affiliation: Affiliations
SINTA Score Overall: 2.326
Subjects: 
Informasi jumlah halaman tidak ditemukan.
